In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import chi2_contingency
from scipy.stats import f_oneway

from optbining import Scorecard, BinningProcess, OptimalBinning
from optbinning.scorecard import plot_auc_roc, plot_cap, plot_ks,ScorecardMonitoring

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score



ModuleNotFoundError: No module named 'optbining'

In [53]:
df = pd.read_excel("DatosPractica_Scoring.xlsx")
#df.drop(columns="ID", inplace=True)

predecir = df.iloc[1285:]
df = df[:1285]

y = df[["Cardhldr", "default"]]
todas_las_columnas = list(df.columns)

for i in ["Cardhldr", "default"]:
    todas_las_columnas.remove(i)


df = df[todas_las_columnas]

In [54]:
#df = pd.concat([df,pd.DataFrame([[0,0,0,0,0,np.nan,0,0,0,0,0]], columns=df.columns)])

In [55]:
df.dtypes

ID            int64
Age         float64
Income      float64
Exp_Inc     float64
Avgexp      float64
Ownrent       int64
Selfempl      int64
Depndt        int64
Inc_per     float64
Cur_add       int64
Major         int64
Active        int64
dtype: object

In [64]:
aceptados = df[y["Cardhldr"]>0.7]
denegados = df[y["Cardhldr"]<=0.7]
y_aceptados = y[y["Cardhldr"]>0.7]["default"]
y_aceptados

0       0.0
1       1.0
3       0.0
4       0.0
6       1.0
       ... 
1278    0.0
1280    0.0
1282    0.0
1283    0.0
1284    1.0
Name: default, Length: 994, dtype: float64

In [56]:
def buscar_nan(df):
    return df[df.isna().any(axis=1)]



In [59]:
def atipicos_rango_intercuartilico(df, variable):
    Q1 = df[variable].quantile(0.25)
    Q3 = df[variable].quantile(0.75)

    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5*IQR
    upper_bound = Q3 + 1.5*IQR

    df = df[(df[variable]<lower_bound) | (df[variable]>upper_bound)]
    return df



def df_join(funcion, df):
    todas_las_columnas = list(df.columns)
    df_atipicos_inter = pd.DataFrame()
    for variable in todas_las_columnas:
        df_atipicios = funcion(df, variable)
        print(df_atipicios.to_string())
        df_atipicos_inter = pd.concat([df_atipicos_inter, df_atipicios])
        print(variable)

    return df_atipicos_inter




## Primera etapa: Construccion del modelo solo con los Aceptados

In [65]:
y_aceptados.value_counts()

default
0.0    890
1.0    104
Name: count, dtype: int64

In [66]:
y_aceptados.value_counts(normalize=True)

default
0.0    0.895372
1.0    0.104628
Name: proportion, dtype: float64

In [67]:
yT_0 = y_aceptados.value_counts(normalize=True)[0]
yT_1 = y_aceptados.value_counts(normalize=True)[1]

In [69]:
aceptados.describe()

,ID,Age,Income,Exp_Inc,Avgexp,Ownrent,Selfempl,Depndt,Inc_per,Cur_add,Major,Active
count,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000
mean,643.062374,33.188715,3.447242,0.089110,239.612725,0.479879,0.061368,0.973843,2.214605,54.691147,0.841046,7.265594
std,366.554739,10.180138,1.716888,0.099877,289.158901,0.499846,0.240125,1.250708,1.355279,64.671143,0.365817,6.074552
min,1.000000,0.166667,0.210000,0.000186,0.000000,0.000000,0.000000,0.000000,0.070000,0.000000,0.000000,0.000000
25%,332.250000,25.333330,2.313625,0.026912,66.823750,0.000000,0.000000,0.000000,1.250000,12.000000,1.000000,2.000000
50%,646.500000,31.000000,3.000000,0.060319,150.540850,0.000000,0.000000,0.000000,1.970667,28.000000,1.000000,6.000000
75%,957.750000,39.729168,4.000000,0.114712,315.023975,1.000000,0.000000,2.000000,2.797500,72.000000,1.000000,11.000000
max,1285.000000,83.500000,13.500000,0.906320,3099.505000,1.000000,1.000000,6.000000,10.999900,540.000000,1.000000,31.000000


### Variables discretas

In [ ]:
aceptados["Ownrent"].value_counts(normalize=True, dropna=False).sort_index()

In [ ]:
ctabla = pd.crosstab(aceptados["Ownrent"], y_aceptados, margins=True).round(3)
ctabla

In [ ]:
c, p, dof, expected = chi2_contingency(ctabla)
print(p)

In [ ]:
pd.crosstab(aceptados["Ownrent"], y_aceptados, margins=True, normalize=True).round(3).plot(figsize=(15,5))
plt.axhline(y=yT_0, color='#1f77b4', linestyle='--')
plt.axhline(y=yT_1, color='#ff7f0e', linestyle='--')
plt.show()

In [ ]:
def valoracion_variables_discretas(df, y, variable):
    ctabla = pd.crosstab(df[variable], y, margins=True).round(3)
    c, p, dof, expected = chi2_contingency(ctabla)
    print(f"pvalue: {p} de la variable {variable}")
    ctabla.plot(figsize=(15,5))
    plt.axhline(y=yT_0, color='#1f77b4', linestyle='--')
    plt.axhline(y=yT_1, color='#ff7f0e', linestyle='--')
    plt.show()


valoracion_variables_discretas(aceptados, y_aceptados, "Ownrent")

### Variables continuas

In [ ]:
def valoracion_variables_continuas(df, variable):    
    sns.displot(x=(df[variable]), kind="kde", fill=True)
    plt.show()
    sns.displot(x=np.log(df[variable]), kind="kde", fill=True)
    plt.show()

In [ ]:
def pvalue_continuas(df, variable):
    fvalue, pvalue = f_oneway(aceptados.loc[y_aceptados==0,[variable]], aceptados.loc[y_aceptados==1,[variable]])
    print(f"para la variable {variable}: ", fvalue, pvalue)

## Selección de variables
### Dividimos la muestra en entrenamiento y test

In [ ]:
dt_train, dt_test = train_test_split(aceptados,stratify=y_aceptados, test_size=.25, random_state=321)

In [ ]:
dt_train["y"].shape

In [ ]:
dt_train["y"].mean()

In [ ]:
dt_test["y"].shape

In [ ]:
dt_test["y"].mean()

### Variables numericas
#### Defino la tramificacion optima

In [ ]:
variable = variable
X=dt_train[variable].values
Y=dt_train['y'].values

In [ ]:
optb = OptimalBinning(name=variable, dtype="numerical", solver="cp")

# Si se quisiese fijar los intervalos manualmente (porque no gusten los que obtine el agoritmo, entonces habría que usar:
#                     user_splits=
#                     user_splits_fixed=
# HAy veces que los datos tienen dátos missing y códigos especiales en este caso para obtener una categoría con esos datos missing y datos especiales hay que establecerlos
#                     special_codes = [-9, -8, -7]

# Una vez definido podemos pasar a estimarlo
optb.fit(X, Y)
optb.splits

In [ ]:
binning_table = optb.binning_table
binning_table.build()

In [ ]:
print("IV= ", binning_table.iv.round(3))
print("Gini= ", binning_table.gini.round(3))

# La última columna muestra el estadístico Jensen-Shannon de divergencia.
# Es una medida de la similaridad entre dos distribuciones de probabilidad (frecuencias de buenos y malos )
# que está acotada entre 0 y log2 (aprox 0.70) (puede utilizarse 0.01 como mínimo) 
print("JS= ", binning_table.js.round(3))


#### Transformacion WoE

In [ ]:
# Transformación WOE
x_woe = optb.transform(X, metric="woe")
pd.Series(x_woe).value_counts().sort_index()

In [ ]:
pd.crosstab(x_woe,Y,normalize=0).round(3)

In [ ]:
fig, ax = plt.subplots()
ax.plot(pd.crosstab(x_woe,Y,normalize=0).iloc[:,1])
ax.set_xlabel("x_woe")
ax.set_ylabel("porcentaje de impago")

In [ ]:
# Transformación WOE en el conjunto test
x_test_woe = optb.transform(dt_test[variable].values, metric="woe")
pd.Series(x_test_woe).value_counts().sort_index()

### Variables categoricas
#### Agrupacion de variables categoricas

In [ ]:
variable_cat = "purpose"
X_cat = dt_train[variable_cat].values
Y_cat = dt_train['y'].values

dt_train[variable_cat].value_counts()

In [ ]:
optb = OptimalBinning(name=variable_cat, dtype="categorical", solver="cp",
                      cat_cutoff=0.1)  # podemos cambiar los valores por defecto cat_cutoff=None, o, cat_cutoff=0.005

optb.fit(X_cat, Y_cat)
optb.splits

In [ ]:
binning_table = optb.binning_table
binning_table.build()

In [ ]:
binning_table.plot(metric="event_rate")

In [ ]:
x_woe = optb.transform(X_cat, metric="woe")
pd.Series(x_woe).value_counts()